[**⬆️ Back to the index notebook**](./00_Index.ipynb)

In [9]:
from IPython.display import HTML
HTML('''<script>code_show=true; function code_toggle() { if (code_show){ $('div.input').hide(); } else { $('div.input').show(); } code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Setup

In [140]:
import json
import pandas as pd
import os
import requests as rq

In [141]:
# set directories
RAW = os.path.join("..","data", "raw")
INTERMEDIATE = os.path.join("..","data", "intermediate")
FINAL = os.path.join("..","data", "final")
REGISTRY_ID = 'gac-amc'

### Data Aquisition

#### Summary / Stats

In [163]:
stats_data = {}

stats_data['timeliness.csv'] = pd.read_csv(
    "http://dashboard.iatistandard.org/timeliness_frequency.csv")
stats_data['comprehensiveness_core.csv'] = pd.read_csv(
    "http://dashboard.iatistandard.org/comprehensiveness_core.csv")
stats_data['comprehensiveness_financials.csv'] = pd.read_csv(
    "http://dashboard.iatistandard.org/comprehensiveness_financials.csv")
stats_data['comprehensiveness_valueadded.csv'] = pd.read_csv(
    "http://dashboard.iatistandard.org/comprehensiveness_valueadded.csv")
# similar approach for any dashboard tables

In [164]:
combined_comprehensiveness = pd.merge(
    on="Publisher Registry Id", 
    left = pd.merge(
        on="Publisher Registry Id", 
        left=stats_data['comprehensiveness_core.csv'], 
        right=stats_data['comprehensiveness_financials.csv']), 
    right=stats_data['comprehensiveness_valueadded.csv'])

combined_comprehensiveness.set_index('Publisher Registry Id', inplace=True)

pd.DataFrame(combined_comprehensiveness.transpose()[REGISTRY_ID])

gac-amc
Publisher Name_x                                    Canada - Global Affairs Canada | Affaires mond...
Version (with valid data)                                                                         100
Reporting-Org (with valid data)                                                                   100
Iati-identifier (with valid data)                                                                 100
Participating Organisation (with valid data)                                                      100
Title (with valid data)                                                                           100
Description (with valid data)                                                                     100
Status (with valid data)                                                                          100
Activity Date (with valid data)                                                                   100
Sector (with valid data)                                                                          100
Country or Region (with valid data)                                                               100
Average (with valid data)_x                                                                       100
Version (with any data)                                                                           100
Reporting-Org (with any data)                                                                     100
Iati-identifier (with any data)                                                                   100
Participating Organisation (with any data)                                                        100
Title (with any data)                                                                             100
Description (with any data)                                                                       100
Status (with any data)                                                                            100
Activity Date (with any data)                                                                     100
Sector (with any data)                                                                            100
Country or Region (with any data)                                                                 100
Average (with any data)_x                                                                         100
Publisher Name_y                                    Canada - Global Affairs Canada | Affaires mond...
Transaction - Commitment (with valid data)                                                        100
Transaction - Disbursement or Expenditure (with...                                                 87
Transaction - Traceability (with valid data)                                                      100
Budget (with valid data)                                                                          100
Average (with valid data)_y                                                                        97
Transaction - Commitment (with any data)                                                          100
Transaction - Disbursement or Expenditure (with...                                                 87
Transaction - Traceability (with any data)                                                        100
Budget (with any data)                                                                            100
Average (with any data)_y                                                                          97
Publisher Name                                      Canada - Global Affairs Canada | Affaires mond...
Contacts (with valid data)                                                                        100
Location Details (with valid data)                                                                 95
Geographic Coordinates (with valid data)                                                           79
DAC Sectors (with valid data)                                                                      98
Capital Spend 

In [115]:
for name, dataset in stats_data.items():
    dataset.to_csv(os.path.join(RAW,name))

#### Raw IATI

In [157]:
# File 1/2 (Active Activities)
request = rq.get("http://w05.international.gc.ca/projectbrowser-banqueprojets/iita-iati/dfatd-maecd_activit_status_2_3.xml")

with open(os.path.join(RAW, "status_2-3.xml"), "wb") as out_file:
  out_file.write(request.content)

In [158]:
# File 2/2 (Completed Activities)
request = rq.get("http://w05.international.gc.ca/projectbrowser-banqueprojets/iita-iati/dfatd-maecd_activit_status_4.xml")

with open(os.path.join(RAW, "status_4.xml"), "wb") as out_file:
  out_file.write(request.content)